# VTK - ITK Project : Etude longitudinale de l'évolution d'une tumeur

### Par Raphael Duhen et Andrews Nigel

In [1]:
# Imports
import vtk
import itk
import matplotlib.pyplot as plt
import numpy as np

In [2]:
reader = vtk.vtkNrrdReader()
reader.SetFileName("Data/case6_gre1.nrrd")

window = vtk.vtkRenderWindow()
renderer = vtk.vtkRenderer()

window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
window.SetInteractor(interactor)

contour = vtk.vtkContourFilter()
contour.SetInputConnection(reader.GetOutputPort())
contour.SetValue(0, 135)

contourMapper = vtk.vtkPolyDataMapper()
contourMapper.SetInputConnection(contour.GetOutputPort())
contourMapper.ScalarVisibilityOff()

contourActor = vtk.vtkActor()
contourActor.SetMapper(contourMapper)

renderer.AddActor(contourActor)

window.Render()
interactor.Start()